In [90]:
import requests
import json
import csv
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import random
import os
import string
from time import sleep
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from tqdm.auto import tqdm
from ratelimit import limits, sleep_and_retry
from dotenv import load_dotenv
import Levenshtein
load_dotenv();
tqdm.pandas()

In [9]:
has_goodreads = pd.read_csv('5_has_goodreads.csv')
typos_fixed = pd.read_csv('6_typos_fixed.csv')
books = pd.read_csv('4_no_null_prosecraft.csv')

In [7]:
len(has_goodreads)

2288

In [8]:
len(typos_fixed)

523

In [10]:
len(books)

24857

In [12]:
len(has_goodreads[has_goodreads['year'].isna()])

531

In [13]:
len(typos_fixed[typos_fixed['year'].isna()])

321

In [38]:
has_more_gr = pd.concat([has_goodreads,typos_fixed]).drop_duplicates(subset=['title','author'],keep='last').drop('Unnamed: 0',axis=1)

In [39]:
len(has_more_gr)

2288

In [40]:
len(has_more_gr[has_more_gr['year'].isna()])

329

In [42]:
has_more_gr.head()

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year
1,Ryan's Christmas,L. J. Ross,41547.0,41.47,8.49,2.84,0.94,1.91,"['Mystery', 'Crime', 'Fiction', 'Christmas', '...",2019.0
2,The Very Best of Caitlin R. Kiernan,Caitlin R. Kiernan,165604.0,62.20,7.67,3.20,0.90,2.30,"['Horror', 'Short Stories', 'Fantasy', 'Fictio...",2019.0
3,Christmas Pig,J. K. Rowling,51218.0,61.10,7.93,3.17,0.81,2.36,"['Fantasy', 'Christmas', 'Fiction', 'Childrens...",2021.0
4,Fatal Love,Michael Patterson,85334.0,18.00,11.14,5.15,2.14,3.01,[],2019.0
7,Wild Hunt,Emma L. Adams,77505.0,42.26,8.35,2.51,0.83,1.68,"['Urban Fantasy', 'Fae']",2021.0


In [46]:
len(has_more_gr[(has_more_gr['genre'] == '[]') & has_more_gr['year'].notna()])

443

In [49]:
len(has_goodreads[(has_goodreads['genre'] == '[]') & (has_goodreads['year'].notna())])

388

In [50]:
len(typos_fixed[(typos_fixed['genre'] == '[]') & typos_fixed['year'].notna()])

55

In [54]:
new_books = pd.concat([books,has_more_gr]).drop_duplicates(subset=['title','author'],keep='last').drop('Unnamed: 0',axis=1)
new_books.head()

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year
0,The Vanished Birds,Simon Jimenez,124205.0,55.18,6.37,1.95,0.36,1.58,"['Science Fiction', 'Fiction', 'Fantasy', 'Que...",2020.0
1,The Price of Honor,Jonathan P. Brazee,77253.0,35.35,8.71,2.63,0.71,1.92,['Science Fiction'],2017.0
2,The Mathematical Murder of Innocence,Michael Carter,37688.0,24.08,8.11,4.13,1.56,2.58,[],2020.0
3,The Case of the Baker Street Irregulars,Anthony Boucher,80557.0,32.33,8.41,3.72,1.64,2.08,"['Mystery', 'Fiction', 'Crime', 'Humor', 'Clas...",1940.0
4,Zombie Nation,Charlie Dalton,64396.0,51.11,8.22,2.21,0.58,1.63,[],2020.0


In [55]:
len(new_books)

25049

In [64]:
new_books = new_books[(new_books['genre'] != '[]') | (new_books['year'].isna())]

In [65]:
len(new_books[new_books['genre'] == '[]']

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year
873,What You Don’t Remember,Kristy Cooper & Steve Johnson,79738.0,40.51,8.71,2.90,0.89,2.01,[],NaN
904,Devil’s Due,Taylor Anderson,172799.0,47.96,7.38,4.07,1.81,2.26,[],NaN
919,"Pan’s Labyrinth, The Labyrinth of the Faun",Guillermo del Toro & Cornelia Funke,53468.0,65.18,7.11,2.71,0.82,1.89,[],NaN
1100,Goosebumps: Why I’m Afraid of Bees,R. L. Stine,20624.0,54.13,7.06,3.39,1.30,2.09,[],NaN
1329,Star Wars: Queen’s Shadow,E. K. Johnston,76298.0,28.82,9.65,3.30,1.07,2.23,[],NaN
...,...,...,...,...,...,...,...,...,...,...
514,The Women,T. C. Boyle,180303.0,58.89,7.01,2.65,0.84,1.81,[],NaN
516,Sam Wilson,Zodiac,96217.0,46.19,8.19,2.27,0.93,1.34,[],NaN
518,The Hardy Boys: Fear On Wheels,Franklin W. Dixon,32416.0,43.88,7.55,2.21,0.66,1.55,[],NaN
521,Siege,Stephen White,119341.0,40.08,9.18,2.17,0.76,1.40,[],NaN


In [68]:
len(books[(books['genre'] == '[]') & books['year'].isna()].drop('Unnamed: 0', axis=1)

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year
19,The Teacher,Michael Ben-Naftali,50233.0,37.06,7.20,2.93,1.25,1.68,[],NaN
20,Ryan's Christmas,L. J. Ross,41547.0,41.47,8.49,2.84,0.94,1.91,[],NaN
53,The Very Best of Caitlin R. Kiernan,Caitlin R. Kiernan,165604.0,62.20,7.67,3.20,0.90,2.30,[],NaN
55,Christmas Pig,J. K. Rowling,51218.0,61.10,7.93,3.17,0.81,2.36,[],NaN
63,Fatal Love,Michael Patterson,85334.0,18.00,11.14,5.15,2.14,3.01,[],NaN
...,...,...,...,...,...,...,...,...,...,...
24838,In Veritas -,C. J. Lavigne,128055.0,82.66,6.83,2.74,1.15,1.59,[],NaN
24839,Star Wars - The New Jedi Order: Vector Prime,R. A. Salvatore,118450.0,42.30,5.46,3.38,1.48,1.90,[],NaN
24840,Blessing in Disguise,Danielle Steel,82682.0,25.47,10.01,3.24,1.03,2.22,[],NaN
24845,How To Have Kick-Ass Ideas,Chris Baréz-Brown,28957.0,26.86,8.99,4.20,1.14,3.06,[],NaN


In [69]:
len(new_books)

22449

In [71]:
len(new_books[new_books['genre'] != '[]'])

21864

In [72]:
len(new_books) - len(new_books[new_books['genre'] != '[]'])

585

In [77]:
# Books I potentially should have information for: Includes books with all information, and books with no years.
len(books[(books['genre'] != '[]') | books['year'].isna()])

22700

In [78]:
# Books I *do* have genre information for
len(books[(books['genre'] != '[]')])

20338

In [80]:
# Books for which genre information *might* still exist that I don't have. 
len(new_books[new_books['year'].isna() & (new_books['genre'] == '[]')])

585

In [81]:
to_retry = new_books[new_books['year'].isna() & (new_books['genre'] == '[]')]

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year
514,The Women,T. C. Boyle,180303.0,58.89,7.01,2.65,0.84,1.81,[],NaN
516,Sam Wilson,Zodiac,96217.0,46.19,8.19,2.27,0.93,1.34,[],NaN
518,The Hardy Boys: Fear On Wheels,Franklin W. Dixon,32416.0,43.88,7.55,2.21,0.66,1.55,[],NaN
521,Siege,Stephen White,119341.0,40.08,9.18,2.17,0.76,1.40,[],NaN
522,The Transformation of Man,J. Krishnamurti,79250.0,14.24,12.09,4.08,1.32,2.75,[],NaN


In [84]:
def get_genres_typos(index): 
    book_id = -1
    itemqueue = []
    row = to_retry.loc[index].copy()
    title = row['title']
    author = row['author'].replace(' & ', ' ')
    last_name = row['author'].split(' & ')[0].split()[-1]
    first_name = row['author'].split(' & ')[0].split()[0]
    possible_error = True
    book_list = list()
    

    browser.get('http://www.goodreads.com/search?q=&qid=')
    try: 
        search_book = browser.find_element(By.ID, value='search_query_main')
    except NoSuchElementException: 
        sleep(random.uniform(3,10))
        browser.get('http://www.goodreads.com/search?q=&qid=')
        search_book = browser.find_element(By.ID, value='search_query_main')
    search_book.send_keys(title + ' ' + author)
    search_book.submit()
    sleep(random.uniform(3,10))
    itemqueue = browser.find_elements(By.XPATH, value="//table/tbody/tr[contains(@itemtype, 'http://schema.org/Book')]")
    img = browser.find_elements(By.CLASS_NAME, value="bookCover")
    
    #Sometimes middle initials mess up the search. Try again with just first and last name.       
    if len(itemqueue) == 0: 
        browser.get('http://www.goodreads.com/search?q=&qid=')
        author = author.split()[0] + ' ' + author.split()[-1]
        try: 
            search_book = browser.find_element(By.ID, value='search_query_main')
        except NoSuchElementException: 
            sleep(random.uniform(3,10))
            browser.get('http://www.goodreads.com/search?q=&qid=')
            search_book = browser.find_element(By.ID, value='search_query_main')
        search_book.send_keys(title + ' ' + author)
        search_book.submit()
        sleep(random.uniform(3,10))
        itemqueue = browser.find_elements(By.XPATH, value="//table/tbody/tr[contains(@itemtype, 'http://schema.org/Book')]")
        img = browser.find_elements(By.CLASS_NAME, value="bookCover")
        sleep(random.uniform(3,10))

    #Try just searching for the last name, see if anything comes up.     
    if len(itemqueue) == 0:
        browser.get('http://www.goodreads.com/search?q=&qid=')
        try: 
            search_book = browser.find_element(By.ID, value='search_query_main')
        except NoSuchElementException: 
            sleep(random.uniform(3,10))
            browser.get('http://www.goodreads.com/search?q=&qid=')
            search_book = browser.find_element(By.ID, value='search_query_main')            
        
        search_book.send_keys(title + ' ' + last_name)
        search_book.submit()
        sleep(random.uniform(3,10))
        itemqueue = browser.find_elements(By.XPATH, value="//table/tbody/tr[contains(@itemtype, 'http://schema.org/Book')]")
        img = browser.find_elements(By.CLASS_NAME, value="bookCover")
        
        if len(itemqueue) == 0:
            try: 
                browser.get('http://www.goodreads.com/search?q=&qid=')
                search_book = browser.find_element(By.ID, value='search_query_main')
            except NoSuchElementException: 
                sleep(random.uniform(3,10))
                browser.get('http://www.goodreads.com/search?q=&qid=')
                search_book = browser.find_element(By.ID, value='search_query_main')
            search_book.send_keys(title + ' ' + first_name)
            search_book.submit()
            sleep(random.uniform(3,10))
            itemqueue = browser.find_elements(By.XPATH, value="//table/tbody/tr[contains(@itemtype, 'http://schema.org/Book')]")
            img = browser.find_elements(By.CLASS_NAME, value="bookCover")
                
    if len(itemqueue) > 0:
        for i in range(len(itemqueue)):
            book_list.append(itemqueue[i].text.split('\n'))
            book_list_ap = list()
        for i in range(0, len(book_list)):
             book_list_ap.append((book_list[i][0],book_list[i][1],img[i].get_property("src")))                
        
        for book in book_list_ap:
            #Check if exact first and last name are found in the author's name
            if last_name in book[1] and first_name in book[1]:
                book_id = book[2].split('/')[-1].split('.')[0]
                possible_error = False
                break
                
            #Check if first and last name with small typos are found in the author's name    
            else:
                lastname_found = False
                firstname_found = False
                for name in book[1].split():
                    if Levenshtein.distance(name, last_name) <= 2:
                        lastname_found = True
                    if Levenshtein.distance(name, first_name) <= 2:
                        firstname_found = True
                if lastname_found and firstname_found:
                    book_id = book[2].split('/')[-1].split('.')[0]
            
        
    if book_id == -1:
        row['genre'] = []
        row['year'] = np.nan
        return row
    
    book_url = f'https://www.goodreads.com/book/show/{book_id}'
    browser.get(book_url)
    genres = browser.find_elements(By.XPATH, value="//span[contains(@class, 'BookPageMetadataSection__genreButton')]")
    try: 
        datestring = browser.find_element(By.CSS_SELECTOR, ".FeaturedDetails > p:nth-child(2)")
        year = int(datestring.text[-4:])
    except NoSuchElementException:
        year = np.nan
    except ValueError:
        year = np.nan
    
    #Keep a running list of possible typos as a list of tuples. 
    if possible_error:
        typo = row['author']
        try: 
            actual = browser.find_element(By.XPATH, value="//span[contains(@class, 'ContributorLink__name')]").text
        except:
            actual = '?'
        possible_errors.append((typo, actual))
        print(f"Typo found! {typo} should be {actual}.")
        
    
    sleep(random.uniform(3,10))
    row['genre'] = [genre.text for genre in genres]
    row['year'] = year
    return row

In [86]:
#Fill in my Goodreads username and password from the .env
user_name = os.environ.get('USER')
password = os.environ.get('PASSWORD')

#This is just the URL I get when I go to goodreads and select log in by email.
login_url = os.environ.get('URL')

In [87]:
#Here I start up a headless Firefox browser through Selenium
s = Service("geckodriver.exe")
opts=Options()
opts.add_argument('-headless')
browser = webdriver.Firefox(service=s)
browser.get(login_url)

In [88]:
# Here I log into goodreads
log_email = browser.find_element(By.ID, value="ap_email")
log_pwd = browser.find_element(By.ID, value="ap_password")
log_email.send_keys(user_name)
log_pwd.send_keys(password)
log_pwd.submit()
sleep(5)

In [92]:
possible_errors = []

In [93]:
next_trial = to_retry.progress_apply(lambda row: get_genres_typos(row.name), axis=1)

  0%|          | 0/585 [00:00<?, ?it/s]

In [94]:
next_trial.head()

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year
873,What You Don’t Remember,Kristy Cooper & Steve Johnson,79738.0,40.51,8.71,2.90,0.89,2.01,[],2021.0
904,Devil’s Due,Taylor Anderson,172799.0,47.96,7.38,4.07,1.81,2.26,"[Alternate History, Science Fiction, Fantasy, ...",2017.0
919,"Pan’s Labyrinth, The Labyrinth of the Faun",Guillermo del Toro & Cornelia Funke,53468.0,65.18,7.11,2.71,0.82,1.89,"[Fantasy, Horror, Young Adult, Fiction, Histor...",2019.0
1100,Goosebumps: Why I’m Afraid of Bees,R. L. Stine,20624.0,54.13,7.06,3.39,1.30,2.09,"[Horror, Fiction, Childrens, Young Adult, Midd...",1994.0
1329,Star Wars: Queen’s Shadow,E. K. Johnston,76298.0,28.82,9.65,3.30,1.07,2.23,"[Star Wars, Science Fiction, Young Adult, Fict...",2019.0


In [98]:
len(next_trial[next_trial['year'].notna()])/len(next_trial)

0.3606837606837607

In [99]:
len(next_trial[(next_trial['genre']==) & next_trial['year'].notna()])

0

In [102]:
next_trial[next_trial['year'].notna()]

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year
873,What You Don’t Remember,Kristy Cooper & Steve Johnson,79738.0,40.51,8.71,2.90,0.89,2.01,[],2021.0
904,Devil’s Due,Taylor Anderson,172799.0,47.96,7.38,4.07,1.81,2.26,"[Alternate History, Science Fiction, Fantasy, ...",2017.0
919,"Pan’s Labyrinth, The Labyrinth of the Faun",Guillermo del Toro & Cornelia Funke,53468.0,65.18,7.11,2.71,0.82,1.89,"[Fantasy, Horror, Young Adult, Fiction, Histor...",2019.0
1100,Goosebumps: Why I’m Afraid of Bees,R. L. Stine,20624.0,54.13,7.06,3.39,1.30,2.09,"[Horror, Fiction, Childrens, Young Adult, Midd...",1994.0
1329,Star Wars: Queen’s Shadow,E. K. Johnston,76298.0,28.82,9.65,3.30,1.07,2.23,"[Star Wars, Science Fiction, Young Adult, Fict...",2019.0
...,...,...,...,...,...,...,...,...,...,...
24847,(R)Evolution,M. E. Purfield,61185.0,51.19,7.25,2.09,0.46,1.63,[],2022.0
222,Moth,James Sallis,56267.0,52.27,7.19,3.37,1.10,2.27,"[Mystery, Noir, Fiction, Crime, Detective, Thr...",1993.0
314,Good Friends,Leanne Moriarty,60361.0,73.86,5.92,2.06,0.63,1.43,[],2020.0
387,True Blue,Jane Smiley,73746.0,50.61,8.61,3.21,0.56,2.65,"[Horses, Young Adult, Historical Fiction, Anim...",2011.0


In [104]:
fixed_books = pd.concat([new_books,next_trial]).drop_duplicates(subset=['title','author'],keep='last').reset_index(drop=True)

In [106]:
fixed_books[fixed_books['year'].isna()]

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year
221,Death of a Fairy Tale,Emily Fluke,72846.0,60.24,5.52,2.17,0.61,1.56,"['Mystery', 'Fantasy', 'Fairy Tales', 'Young A...",NaN
402,Matchless,Gregory Maguire,2792.0,82.70,5.55,2.51,0.36,2.15,"['Christmas', 'Fiction', 'Fantasy', 'Fairy Tal...",NaN
693,Kheris Burning,C. G. Hatton,60115.0,47.33,8.82,2.12,0.54,1.58,"['Science Fiction', 'Fiction']",NaN
850,Erec Rex: The Search for Truth,Kaza Kingsley,126688.0,50.62,8.75,3.12,0.68,2.44,"['Fantasy', 'Young Adult', 'Magic', 'Fiction',...",NaN
1530,The Truth Will Out,Rosemary Hennigan,98047.0,47.57,7.92,3.06,1.23,1.83,"['Fiction', 'Thriller', 'Mystery', 'Irish Lite...",NaN
...,...,...,...,...,...,...,...,...,...,...
22444,The Women,T. C. Boyle,180303.0,58.89,7.01,2.65,0.84,1.81,[],NaN
22445,Sam Wilson,Zodiac,96217.0,46.19,8.19,2.27,0.93,1.34,[],NaN
22446,The Hardy Boys: Fear On Wheels,Franklin W. Dixon,32416.0,43.88,7.55,2.21,0.66,1.55,[],NaN
22447,Siege,Stephen White,119341.0,40.08,9.18,2.17,0.76,1.40,[],NaN


In [112]:
len(fixed_books[fixed_books['year'].isna()])

465

In [113]:
len(has_goodreads[has_goodreads['year'].isna()])

531

In [115]:
fixed_books.to_csv('7_final_books.csv')